This script is designed to schedule a time-series microcosm experiment in which all microcosms are destructively sampled at the same time, rather than sequentially from t=0. 

This script must be run from a python2.7 conda environment.

REQUIREMENTS
    
    gcalcli    https://github.com/insanum/gcalcli
    fortune    sudo apt-get install fortune
    
    

In [1]:
import os, re, sys, getopt, datetime, subprocess
from dateutil.relativedelta import relativedelta

In [2]:
## OBTAIN START DATE INFORMATION FROM USER
def ObtainStart():
    isValid=False
    
    userDate = "28/02/17"
    userTime = "9:30"
    
    while not isValid:
        #userDate = raw_input("Enter Start Date (dd/mm/yy): ")
        #userTime = raw_input("Enter Start Time (HH:MM): ")
        
        try: # strptime throws an exception if the input doesn't match the pattern
            d = datetime.datetime.strptime(userDate, "%d/%m/%y")
            t = datetime.datetime.strptime(userTime, "%H:%M")
            c = datetime.datetime.combine(d.date(), t.time())
            
            isValid=True
        except:
            print "Doh, try again!\n"
    return c

In [3]:
## OBTAIN TIMEPOINT INFORMATION
def ObtainTimePoints():
    isValid=False
    
    while not isValid:
        #points = raw_input("Enter Time Points (4,7,14,32,48): ")
        #unit = raw_input("Enter Unit of Time (minutes | hours | days | months): ")
        
        timepoints = ['12','24','36']
        unit = 'hours'
        
        try: # strptime throws an exception if the input doesn't match the pattern
            #points = points.strip()
            #timepoints = points.split(",")
            labels = timepoints
            
            print "You have scheduled "+str(len(timepoints))+" timepoints, on the time-scale of "+unit
            isValid=True
        except:
            print "Doh, try again!\n"
    
    timepoints = map(int, timepoints) #Convert strings to integers

    ## Standardize units to hours
    if unit == "minutes":
        timepoints = [x / 60 for x in timepoints]
        
    elif unit == "days":
        timepoints = [x * 24 for x in timepoints]
        
    elif unit == "months":
        timepoints = [x * 24 * 31 for x in timepoints]
        
    return timepoints,labels, unit

In [4]:
## DETERMINE SCHEDULE
def Schedule(start, timepoints):
    # Determine pre-incubation length
    
    #preincubation = int(raw_input("Please Specify the Number of days for Pre-incubation (0 if none): "))
    preincubation = 0
    
    # Calculate End Date
    END_DATE = start + relativedelta(hours=+max(timepoints)) 

    ## Add preincubation time
    END_DATE = END_DATE + relativedelta(days=+preincubation) 
    
    preincubate_list = []
    substrate_list = []
    
    for time in timepoints:
        if preincubation != 0:
            preincubate_list.append(END_DATE - relativedelta(hours=time) - relativedelta(days=preincubation))
            substrate_list.append(END_DATE - relativedelta(hours=time))
        else:
            substrate_list.append(END_DATE - relativedelta(hours=time))
            
    return END_DATE, preincubate_list, substrate_list

In [5]:
## Add Events to Google Calendar
def Add2Calendar(end, preincubate, ammend, unit):
  
    # Grab Quote to Inspire
    quote = subprocess.check_output('fortune')

    # Add End Date
    print(' '.join([
        "gcalcli",
        "--calendar",
        '\'Roli Wilhelm\'',
        "--title",
        '\'END OF MICROCOSM EXPERIMENT\'',
        "--when",
        '\''+str(end)+'\'',
        "--reminder",
        '\'60\'',
        "--where",
        '\'708 Bradfield\'',
        "--duration",
        '\'180\'',
        "--description",
        '\'Lyophilize\'',
        "add"
    ]))    

In [6]:
def main():
    start = ObtainStart()
    timepoints,labels,unit = ObtainTimePoints()
    end, preincubate, ammend = Schedule(start, timepoints)
    print(Add2Calendar(end, preincubate, ammend, unit))
    
    print("Preincubate Times")
    for i in preincubate:
        print(i)
        
    print("Substrate Addition Times")
    for i in ammend:
        print(i)
    
    print("End Date")
    print(end)    

In [7]:
main()

You have scheduled 3 timepoints, on the time-scale of hours
gcalcli --calendar 'Roli Wilhelm' --title 'END OF MICROCOSM EXPERIMENT' --when 2017-03-01 21:30:00 --reminder '60' --where '708 Bradfield' --duration '180' --description 'Lyophilize' add
None
Preincubate Times
Substrate Addition Times
2017-03-01 09:30:00
2017-02-28 21:30:00
2017-02-28 09:30:00
End Date
2017-03-01 21:30:00
